# Making Your Model Learn Addition!
## Task 1: Introduction

Given the string "54+7", the model should return a prediction: "61".

In [1]:
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import TimeDistributed, Dense, Dropout, SimpleRNN, RepeatVector
from tensorflow.keras.callbacks import EarlyStopping, LambdaCallback

from termcolor import colored

## Task 2: Generate Data
___
Note: If you are starting the notebook from this task, you can run cells from all previous tasks in the kernel by going to the top menu and then selecting Kernel > Restart and Run All
___

In [3]:
all_char = '0123456789+'
# Convert these char to 1-hot encoded list
no_features = len(all_char)
print("Number of features = ",no_features)

Number of features =  11


In [4]:
char_to_index = dict((key, i) for i,key in enumerate(all_char))
index_to_char = dict((i, key) for i, key in enumerate(all_char))

In [6]:
def gen_data():
    first = np.random.randint(0,100)
    second = np.random.randint(0,100)
    example = str(first)+'+'+str(second)
    label = str(first+second)
    return example, label
    
gen_data()

('91+96', '187')

## Task 3: Create the Model
___
Note: If you are starting the notebook from this task, you can run cells from all previous tasks in the kernel by going to the top menu and then selecting Kernel > Restart and Run All
___
Consider these two reviews:

Review 1: This movie is not terrible at all.

Review 2: This movie is pretty decent.

In [9]:
# RNN allow us to input varied length features and labels
# Eg: Sentiment analysis
# Simple RNN is a FC RNN, /p is feb back as i/p
# activation used = 'tanh'
hidden_units = 128
# max length of the expression will be 5 units long | '99+99'
max_time_steps = 5   # Time steps may vary
model = Sequential([SimpleRNN(hidden_units, input_shape=(None, no_features)),
                   RepeatVector(max_time_steps),
                   # Encoder^ | Decoder:
                   SimpleRNN(hidden_units, return_sequences=True),
                   TimeDistributed(Dense(no_features, activation='softmax'))])
# input shape will be const as no_features is const, batch_size may vary
model.compile(loss='categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_2 (SimpleRNN)     (None, 128)               17920     
_________________________________________________________________
repeat_vector_2 (RepeatVecto (None, 5, 128)            0         
_________________________________________________________________
simple_rnn_3 (SimpleRNN)     (None, 5, 128)            32896     
_________________________________________________________________
time_distributed (TimeDistri (None, 5, 11)             1419      
Total params: 52,235
Trainable params: 52,235
Non-trainable params: 0
_________________________________________________________________


## Task 4: Vectorize and De-Vectorize Data
___
Note: If you are starting the notebook from this task, you can run cells from all previous tasks in the kernel by going to the top menu and then selecting Kernel > Restart and Run All
___

In [12]:
def vectorize_eg(example, label):
    x=np.zeros((max_time_steps, no_features))
    y=np.zeros((max_time_steps,no_features))
    # Here we padded with 0's to make them of the same length as time steps is ariable
    
    diffx=max_time_steps-len(example)
    diffy=max_time_steps-len(label)
    
    # Now 1-hot encode
    for i, key in enumerate(example):
        x[i+diffx, char_to_index[key]] = 1
    for i in range(diffx):
        # Hardcoding the positional value
        x[i, char_to_index['0']] = 1
    for i, key in enumerate(label):
        y[i+diffy, char_to_index[key]] = 1
    for i in range(diffy):
        y[i, char_to_index['0']] = 1
    return x,y
e,l = gen_data()
print(e,l)
x,y = vectorize_eg(e,l)
print(x.shape,y.shape)

25+79 104
(5, 11) (5, 11)


In [14]:
def devectorize_eg(example):
    # index2char dict used convert index to char
    # argmax to get the index with max value
    result = [index_to_char[np.argmax(vec)] for i, vec in enumerate(example)]
    # outputs a list of char. Join all of them to form a string
    return ''.join(result)
devectorize_eg(x)

'25+79'

In [15]:
devectorize_eg(y)
# As we see here, y has been padded with 0's

'00104'

## Task 5: Create Dataset
___
Note: If you are starting the notebook from this task, you can run cells from all previous tasks in the kernel by going to the top menu and then selecting Kernel > Restart and Run All
___

## Task 6: Training the Model
___
Note: If you are starting the notebook from this task, you can run cells from all previous tasks in the kernel by going to the top menu and then selecting Kernel > Restart and Run All
___